In [68]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import pickle as pkl

from getEmbeddings import getEmbeddings
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from skorch import NeuralNetClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('..')
import transform_wrappers
device = 'cuda'

class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(5000, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 256)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(256, 80)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(80, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = F.softmax(x, dim=1)
        return x

# import data

In [57]:
data_dir_path = './datasets'

# Import `fake_or_real_news.csv`
df = pd.read_csv(data_dir_path + "/train.csv")

# Set `y`
X = df['text']
y = df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
df.head()

# must use raw term counts for LDA because it is a probabilistic graphical model
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=5000, stop_words='english')

# fit and transform the training data into bag of words
bag_of_words_train = count_vectorizer.fit_transform(X_train.astype('U'))
bag_of_words_test = count_vectorizer.transform(X_test.astype('U'))
feature_names = count_vectorizer.get_feature_names()

# run/load lda

In [59]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx), end=' ')
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))



# Run LDA
num_topics = 10
# lda = LatentDirichletAllocation(n_components=num_topics, random_state=42).fit(bag_of_words_train)
# pkl.dump(lda, open('lda_10.pkl', 'wb'))
lda = pkl.load(open('lda_10.pkl', 'rb'))
print(lda.components_.shape)

# display        
num_top_words = 10
display_topics(lda, feature_names, num_top_words)        

(10, 5000)
Topic 0: said percent new year company million 000 money years companies
Topic 1: mr said trump president ms court new mrs campaign house
Topic 2: trump president obama donald people house election said party white
Topic 3: said police people state city syria attack officers killed military
Topic 4: news twitter com 2016 media facebook 2017 breitbart video на
Topic 5: people world black israel political state women war america students
Topic 6: like just people time don know way life make good
Topic 7: russia united states government russian war foreign china president military
Topic 8: said mr ms new la like york year city years
Topic 9: clinton hillary election campaign fbi trump emails investigation comey email


# load embeddings
Previously was using from doc2vec to embed each document as a 300-dim vector.

In [ ]:
'''
print('getting doc2vec embdeddings...')
xtr, xte, ytr, yte = getEmbeddings("datasets/train.csv")
np.save('./xtr', xtr)
np.save('./xte', xte)
np.save('./ytr', ytr)
np.save('./yte', yte)

# prepare data
xtr = np.load('./xtr.npy').astype(np.float32)
xte = np.load('./xte.npy').astype(np.float32)
ytr = np.load('./ytr.npy')
yte = np.load('./yte.npy')

# encode the labels
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y = np_utils.to_categorical((label_encoder.transform(y_train))).astype(np.int64)
encoded_y_test = np_utils.to_categorical((label_encoder.transform(y_test))).astype(np.int64)
'''

In [75]:
# fit model
net = NeuralNetClassifier(
    FNN,
    max_epochs=10,
    lr=0.01,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    train_split=None,
)
net.fit(bag_of_words_train.astype(np.float32), y_train)
print("Model Trained!")
pkl.dump(net, open('model.pkl', 'wb'))

  epoch    train_loss     dur
-------  ------------  ------
      1        0.6767  5.9020
      2        0.5927  6.0032
      3        0.4771  5.8831
      4        0.3837  5.8206
      5        0.3102  5.8309
      6        0.2517  5.8319
      7        0.2063  6.0850
      8        0.1706  6.0844
      9        0.1443  5.8364
     10        0.1236  5.8703
Model Trained!


# load + interpret the model

In [78]:
# evaluate model
net = pkl.load(open('model.pkl', 'rb'))
probabs = net.predict_proba(bag_of_words_test.astype(np.float32))
y_pred = np.argmax(probabs, axis=1)
acc = np.mean(y_pred == y_test)
print(f"acc: {acc:0.2f}")

acc: 0.95


In [79]:
fnn = net.module_
print(fnn)

FNN(
  (fc1): Linear(in_features=5000, out_features=256, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=256, out_features=80, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=80, out_features=2, bias=True)
)


In [34]:
D = lda.components_
print(D.shape)

(10, 5000)


In [20]:
transform = transform_wrappers.lay_from_w(D)
# norm = transform_wrappers.NormLayer(mu=0.1307, std=0.3081)
# reshape = transform_wrappers.ReshapeLayer(shape=(1, 28, 28))
net = transform_wrappers.Net_with_transform(fnn, 
                                            transform=transform).to(device), 
#                                             reshape=reshape,
                                            use_logits=True).to(device)
# print(list(net.modules()))

In [ ]:
x = x_test[0:1]
x.shape
lda.transform(x)

In [22]:
net(x)

AttributeError: 'numpy.ndarray' object has no attribute 'dim'